In [1]:
from csgo.parser import DemoParser
import numpy
import csv
import pandas
import pyserialem
import os
from tabulate import tabulate

In [2]:
def getDemoPaths(dir_path):
    demoPaths = []
    x = 0
    for filename in os.listdir(dir_path):
        demoPaths.append(os.path.join(dir_path, filename))
    return demoPaths

In [3]:
def generateDemoParsers(paths):
    demo_parsers = []
    for x in paths:
        demoParser = DemoParser(demofile=x, demo_id = x.split('/')[-1].split('.')[0].split('_')[0], parse_rate=128)
        demo_parsers.append(demoParser)
    print("# Demo Parsers: " + str(len(demo_parsers)))
    return demo_parsers

In [4]:
def generateData(parsers):
    killData, roundData, damageData = [], [], [],
    for x in parsers:
        x.parse()
        killData.append(x._parse_kills(return_type="df"))
        roundData.append(x._parse_rounds(return_type="df"))
        damageData.append(x._parse_damages(return_type="df"))
    return (killData, roundData, damageData)

In [5]:
def concat_data(aggregate_data):
    final_df = None
    for x in aggregate_data:
        final_df = pandas.concat([final_df, x], ignore_index=True)
    return final_df

In [75]:
def createSidedCSVDF(players, damageData, killData, roundData):
    locationNames = []
    playerNames = []

    Kills = []
    Deaths = []
    KdDifferential = []
    DamageDealt = []
    DamageTaken = []
    DamageDifferential = []
    sides = []
    totalRounds = []
    dataframe = {}
    for x in players.keys():
        for y in killData[killData['AttackerName'].isin(players[x]) | killData['VictimName'].isin(players[x])].VictimAreaName.value_counts().index:
            
            tempCTKills = len(killData[killData['AttackerName'].isin(players[x])][killData['AttackerAreaName'] == y][killData['AttackerSide'] == 'CT'])
            tempCTDeaths = len(killData[killData['VictimName'].isin(players[x])][killData['VictimAreaName'] == y][killData['VictimSide'] == 'CT'])
            tempCTKdDifferential = tempCTKills - tempCTDeaths
            
            tempCTDamageDealt = damageData[damageData['AttackerName'].isin(players[x])][damageData['AttackerAreaName'] == y][damageData['AttackerSide'] == 'CT'].HpDamageTaken.sum()
            tempCTDamageTaken = damageData[damageData['VictimName'].isin(players[x])][damageData['VictimAreaName'] == y][damageData['VictimSide'] == 'CT'].HpDamageTaken.sum()
            tempCTDamageDifferential = tempCTDamageDealt - tempCTDamageTaken
                                                                                                             
            tempTKills = len(killData[killData['AttackerName'].isin(players[x])][killData['AttackerAreaName'] == y][killData['AttackerSide'] == 'T'])
            tempTDeaths = len(killData[killData['VictimName'].isin(players[x])][killData['VictimAreaName'] == y][killData['VictimSide'] == 'T'])
            tempTKdDifferential = tempTKills - tempTDeaths
                                                                                                             
            tempTDamageDealt = damageData[damageData['AttackerName'].isin(players[x])][damageData['AttackerAreaName'] == y][damageData['AttackerSide'] == 'T'].HpDamageTaken.sum()
            tempTDamageTaken = damageData[damageData['VictimName'].isin(players[x])][damageData['VictimAreaName'] == y][damageData['VictimSide'] == 'T'].HpDamageTaken.sum()
            tempTDamageDifferential = tempTDamageDealt - tempTDamageTaken
            
            tempTotalRounds = 0
                                                    
            for z in killData[killData.AttackerName.isin(players[x])].MatchId.value_counts().index:
                tempTotalRounds += roundData[roundData['MatchId'] == z].iloc[-1].RoundNum
            
            locationNames.append(y)
            locationNames.append(y)
            
            playerNames.append(x)
            playerNames.append(x)
            
            Kills.append(tempCTKills)
            Kills.append(tempTKills)
            
            
            Deaths.append(tempCTDeaths)
            Deaths.append(tempTDeaths)
            
            
            KdDifferential.append(tempCTKdDifferential)
            KdDifferential.append(tempTKdDifferential)
            
            
            DamageDealt.append(tempCTDamageDealt)
            DamageTaken.append(tempCTDamageTaken)
            DamageDifferential.append(tempCTDamageDifferential)
            
            DamageDealt.append(tempTDamageDealt)
            DamageTaken.append(tempTDamageTaken)
            DamageDifferential.append(tempTDamageDifferential)
            
            sides.append('CT')
            sides.append('T')
            
            
            totalRounds.append(tempTotalRounds)
            totalRounds.append(tempTotalRounds)
    
    dataframe['Location'] = locationNames
    dataframe['Player'] = playerNames
    dataframe['Kills'] = Kills
    dataframe['Deaths'] = Deaths
    dataframe['K/D'] = KdDifferential
    dataframe['Damage Dealt'] = DamageDealt
    dataframe['Damage Taken'] = DamageTaken
    dataframe['Damage Differential'] = DamageDifferential
    dataframe['Side'] = sides
    dataframe['Total Rounds'] = totalRounds
    
    return pandas.DataFrame.from_dict(dataframe, orient='columns')
                        
                

In [9]:
def createDifferentialDF(players, damageData, killData, roundData):
    locationNames = []
    playerNames = []
    totalKills = []
    totalDeaths = []
    totalKdDifferential = []
    totalDamageDealt = []
    totalDamageTaken = []
    totalDamageDifferential = []
    ctKills = []
    ctDeaths = []
    ctKdDifferential = []
    ctDamageDealt = []
    ctDamageTaken = []
    ctDamageDifferential = []
    tKills = []
    tDeaths = []
    tKdDifferential = []
    tDamageDealt = []
    tDamageTaken = []
    tDamageDifferential = []
    totalRounds = []
    dataframe = {}
    for x in players.keys():
        for y in killData[killData['AttackerName'].isin(players[x]) | killData['VictimName'].isin(players[x])].VictimAreaName.value_counts().index:
            tempTotalKills = len(killData[killData['AttackerName'].isin(players[x])][killData['AttackerAreaName'] == y])
            tempTotalDeaths = len(killData[killData['VictimName'].isin(players[x])][killData['VictimAreaName'] == y])
            tempTotalKdDifferential = tempTotalKills - tempTotalDeaths
            
            tempTotalDamageDealt = damageData[damageData['AttackerName'].isin(players[x])][damageData['AttackerAreaName'] == y].HpDamageTaken.sum()
            tempTotalDamageTaken = damageData[damageData['VictimName'].isin(players[x])][damageData['VictimAreaName'] == y].HpDamageTaken.sum()
            tempTotalDamageDifferential = tempTotalDamageDealt - tempTotalDamageTaken
            
            tempCTKills = len(killData[killData['AttackerName'].isin(players[x])][killData['AttackerAreaName'] == y][killData['AttackerSide'] == 'CT'])
            tempCTDeaths = len(killData[killData['VictimName'].isin(players[x])][killData['VictimAreaName'] == y][killData['VictimSide'] == 'CT'])
            tempCTKdDifferential = tempCTKills - tempCTDeaths
            
            tempCTDamageDealt = damageData[damageData['AttackerName'].isin(players[x])][damageData['AttackerAreaName'] == y][damageData['AttackerSide'] == 'CT'].HpDamageTaken.sum()
            tempCTDamageTaken = damageData[damageData['VictimName'].isin(players[x])][damageData['VictimAreaName'] == y][damageData['VictimSide'] == 'CT'].HpDamageTaken.sum()
            tempCTDamageDifferential = tempCTDamageDealt - tempCTDamageTaken
                                                                                                             
            tempTKills = len(killData[killData['AttackerName'].isin(players[x])][killData['AttackerAreaName'] == y][killData['AttackerSide'] == 'T'])
            tempTDeaths = len(killData[killData['VictimName'].isin(players[x])][killData['VictimAreaName'] == y][killData['VictimSide'] == 'T'])
            tempTKdDifferential = tempTKills - tempTDeaths
                                                                                                             
            tempTDamageDealt = damageData[damageData['AttackerName'].isin(players[x])][damageData['AttackerAreaName'] == y][damageData['AttackerSide'] == 'T'].HpDamageTaken.sum()
            tempTDamageTaken = damageData[damageData['VictimName'].isin(players[x])][damageData['VictimAreaName'] == y][damageData['VictimSide'] == 'T'].HpDamageTaken.sum()
            tempTDamageDifferential = tempTDamageDealt - tempTDamageTaken
            
            tempTotalRounds = 0
                                                    
            for z in killData[killData.AttackerName.isin(players[x])].MatchId.value_counts().index:
                tempTotalRounds += roundData[roundData['MatchId'] == z].iloc[-1].RoundNum
            
            locationNames.append(y)
            playerNames.append(x)
            totalKills.append(tempTotalKills)
            totalDeaths.append(tempTotalDeaths)
            totalKdDifferential.append(tempTotalKdDifferential)
            
            totalDamageDealt.append(tempTotalDamageDealt)
            totalDamageTaken.append(tempTotalDamageTaken)
            totalDamageDifferential.append(tempTotalDamageDifferential)
            
            ctKills.append(tempCTKills)
            ctDeaths.append(tempCTDeaths)
            ctKdDifferential.append(tempCTKdDifferential)
            
            ctDamageDealt.append(tempCTDamageDealt)
            ctDamageTaken.append(tempCTDamageTaken)
            ctDamageDifferential.append(tempCTDamageDifferential)
            
            tKills.append(tempTKills)
            tDeaths.append(tempTDeaths)
            tKdDifferential.append(tempTKdDifferential)
            
            tDamageDealt.append(tempTDamageDealt)
            tDamageTaken.append(tempTDamageTaken)
            tDamageDifferential.append(tempTDamageDifferential)
            
            totalRounds.append(tempTotalRounds)
    
    dataframe['Location'] = locationNames
    dataframe['Player'] = playerNames
    dataframe['Total Kills'] = totalKills
    dataframe['Total Deaths'] = totalDeaths
    dataframe['K/D'] = totalKdDifferential
    dataframe['Total Damage Dealt'] = totalDamageDealt
    dataframe['Total Damage Taken'] = totalDamageTaken
    dataframe['Total Damage Differential'] = totalDamageDifferential
    dataframe['CT Kills'] = ctKills
    dataframe['CT Deaths'] = ctDeaths
    dataframe['CT K/D'] = ctKdDifferential
    dataframe['CT Damage Dealt'] = ctDamageDealt
    dataframe['CT Damage Taken'] = ctDamageTaken
    dataframe['CT Damage Differential'] = ctDamageDifferential
    dataframe['T Kills'] = tKills
    dataframe['T Deaths'] = tDeaths
    dataframe['T K/D'] = tKdDifferential
    dataframe['T Damage Dealt'] = tDamageDealt
    dataframe['T Damage Taken'] = tDamageTaken
    dataframe['T Damage Differential'] = tDamageDifferential
    dataframe['Total Rounds'] = totalRounds
    
    return pandas.DataFrame.from_dict(dataframe, orient='columns')
                        
                

In [10]:
bigPath = "./demos/mirage/Big/Mirage"
vpPath = "./demos/mirage/VP/Mirage"
natusPath = "./demos/mirage/Natus/Mirage"
heroicPath = "./demos/mirage/Heroic/Mirage"
mirageMappingPath = './mappings/mappings/'

In [11]:
liquidPath = "./demos/mirage/Liquid/Mirage"
furiaPath = "./demos/mirage/Furia/Mirage"
complexityPath = "./demos/mirage/Complexity/Mirage"
g2Path = "./demos/mirage/G2/Mirage"

In [29]:
big_output_path = './damageDifferentials/mirage_six_months/sidedData/bigDamageDifferentials.csv'
vp_output_path = './damageDifferentials/mirage_six_months/sidedData/vpDamageDifferentials.csv'
natus_output_path = './damageDifferentials/mirage_six_months/sidedData/natusDamageDifferentials.csv'
heroic_output_path = './damageDifferentials/mirage_six_months/sidedData/heroicDamageDifferentials.csv'

In [30]:
liquid_output_path = './damageDifferentials/mirage_six_months/sidedData/liquidDamageDifferentials.csv'
furia_output_path = './damageDifferentials/mirage_six_months/sidedData/furiaDamageDifferentials.csv'
complexity_output_path = './damageDifferentials/mirage_six_months/sidedData/complexityDamageDifferentials.csv'
g2_output_path = './damageDifferentials/mirage_six_months/sidedData/g2DamageDifferentials.csv'

In [14]:
mappingFile = os.path.join(mirageMappingPath, os.listdir(mirageMappingPath)[0])

In [15]:
file = open(mappingFile, 'r')
lines = file.readlines()
file.close()

In [16]:
for x in range(len(lines)):
    if(lines[x][-1] == '\n'):
        lines[x] = lines[x][:-1]

In [17]:
mappedPlaceNames = {}
for x in lines:
    if(x != ''):
            mappedPlaceNames[x.split(':')[0]] = x.split(':')[1]

In [18]:
bigDemos = getDemoPaths(bigPath)
vpDemos = getDemoPaths(vpPath)
natusDemos = getDemoPaths(natusPath)
heroicDemos = getDemoPaths(heroicPath)

In [19]:
liquidDemos = getDemoPaths(liquidPath)
furiaDemos = getDemoPaths(furiaPath)
complexityDemos = getDemoPaths(complexityPath)
g2Demos = getDemoPaths(g2Path)

In [20]:
bigParsers = generateDemoParsers(bigDemos)
vpParsers = generateDemoParsers(vpDemos)
natusParsers = generateDemoParsers(natusDemos)
heroicParsers = generateDemoParsers(heroicDemos)

20:25:08 [INFO] Go version>=1.14.0
20:25:08 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/Big/Mirage/big-vs-complexity-m1-mirage.dem
20:25:08 [INFO] Setting demo id to big-vs-complexity-m1-mirage
20:25:08 [INFO] Setting parse rate to 128
20:25:08 [INFO] Go version>=1.14.0
20:25:08 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/Big/Mirage/big-vs-envy-m2-mirage.dem
20:25:08 [INFO] Setting demo id to big-vs-envy-m2-mirage
20:25:08 [INFO] Setting parse rate to 128
20:25:08 [INFO] Go version>=1.14.0
20:25:08 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/Big/Mirage/big-vs-funplus-phoenix-m2-mirage.dem
20:25:08 [INFO] Setting demo id to big-vs-funplus-phoenix-m2-mirage
20:25:08 [INFO] Setting parse rate to 128
20:25:09 [INFO] Go version>=1.14.0
20:25:09 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/Big/Mirage/big-vs-hellraisers-

# Demo Parsers: 15


20:25:10 [INFO] Go version>=1.14.0
20:25:10 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/VP/Mirage/virtus-pro-vs-complexity-m1-mirage.dem
20:25:10 [INFO] Setting demo id to virtus-pro-vs-complexity-m1-mirage
20:25:10 [INFO] Setting parse rate to 128
20:25:10 [INFO] Go version>=1.14.0
20:25:10 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/VP/Mirage/virtus-pro-vs-forze-m1-mirage.dem
20:25:10 [INFO] Setting demo id to virtus-pro-vs-forze-m1-mirage
20:25:10 [INFO] Setting parse rate to 128
20:25:10 [INFO] Go version>=1.14.0
20:25:10 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/VP/Mirage/virtus-pro-vs-furia-m2-mirage.dem
20:25:10 [INFO] Setting demo id to virtus-pro-vs-furia-m2-mirage
20:25:10 [INFO] Setting parse rate to 128
20:25:10 [INFO] Go version>=1.14.0
20:25:10 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/VP/Mirage

# Demo Parsers: 8


20:25:11 [INFO] Go version>=1.14.0
20:25:11 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/Natus/Mirage/gambit-vs-natus-vincere-m3-mirage.dem
20:25:11 [INFO] Setting demo id to gambit-vs-natus-vincere-m3-mirage
20:25:11 [INFO] Setting parse rate to 128
20:25:11 [INFO] Go version>=1.14.0
20:25:11 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/Natus/Mirage/natus-vincere-vs-furia-m1-mirage.dem
20:25:11 [INFO] Setting demo id to natus-vincere-vs-furia-m1-mirage
20:25:11 [INFO] Setting parse rate to 128
20:25:11 [INFO] Go version>=1.14.0
20:25:11 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/Natus/Mirage/natus-vincere-vs-g2-m2-mirage.dem
20:25:11 [INFO] Setting demo id to natus-vincere-vs-g2-m2-mirage
20:25:11 [INFO] Setting parse rate to 128
20:25:11 [INFO] Go version>=1.14.0
20:25:11 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mir

# Demo Parsers: 7


20:25:11 [INFO] Go version>=1.14.0
20:25:11 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/Heroic/Mirage/fnatic-vs-heroic-m1-mirage_62175.dem
20:25:11 [INFO] Setting demo id to fnatic-vs-heroic-m1-mirage
20:25:11 [INFO] Setting parse rate to 128
20:25:11 [INFO] Go version>=1.14.0
20:25:11 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/Heroic/Mirage/furia-vs-heroic-m1-mirage_61768.dem
20:25:11 [INFO] Setting demo id to furia-vs-heroic-m1-mirage
20:25:11 [INFO] Setting parse rate to 128
20:25:11 [INFO] Go version>=1.14.0
20:25:11 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/Heroic/Mirage/gambit-vs-heroic-m3-mirage_62447.dem
20:25:11 [INFO] Setting demo id to gambit-vs-heroic-m3-mirage
20:25:11 [INFO] Setting parse rate to 128
20:25:11 [INFO] Go version>=1.14.0
20:25:11 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/Heroic/Mi

# Demo Parsers: 14


In [21]:
complexityDemos

['./demos/mirage/Complexity/Mirage/big-vs-complexity-m1-mirage.dem',
 './demos/mirage/Complexity/Mirage/complexity-vs-ence-m3-mirage.dem',
 './demos/mirage/Complexity/Mirage/complexity-vs-evil-geniuses-m2-mirage.dem',
 './demos/mirage/Complexity/Mirage/complexity-vs-fnatic-m2-mirage.dem',
 './demos/mirage/Complexity/Mirage/complexity-vs-g2-m1-mirage.dem',
 './demos/mirage/Complexity/Mirage/complexity-vs-movistar-riders-m2-mirage.dem',
 './demos/mirage/Complexity/Mirage/complexity-vs-natus-vincere-m2-mirage.dem',
 './demos/mirage/Complexity/Mirage/complexity-vs-nip-m1-mirage.dem',
 './demos/mirage/Complexity/Mirage/complexity-vs-og-m2-mirage.dem',
 './demos/mirage/Complexity/Mirage/complexity-vs-sprout-m1-mirage.dem',
 './demos/mirage/Complexity/Mirage/complexity-vs-vitality-m2-mirage.dem',
 './demos/mirage/Complexity/Mirage/faze-vs-complexity-m3-mirage.dem',
 './demos/mirage/Complexity/Mirage/fnatic-vs-complexity-m2-mirage.dem',
 './demos/mirage/Complexity/Mirage/furia-vs-complexity-m1

In [22]:
liquidParsers = generateDemoParsers(liquidDemos)
furiaParsers = generateDemoParsers(furiaDemos)
complexityParsers = generateDemoParsers(complexityDemos)
g2Parsers = generateDemoParsers(g2Demos)

20:25:12 [INFO] Go version>=1.14.0
20:25:12 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/Liquid/Mirage/faze-vs-liquid-m1-mirage.dem
20:25:12 [INFO] Setting demo id to faze-vs-liquid-m1-mirage
20:25:12 [INFO] Setting parse rate to 128
20:25:12 [INFO] Go version>=1.14.0
20:25:12 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/Liquid/Mirage/furia-vs-liquid-m3-mirage.dem
20:25:12 [INFO] Setting demo id to furia-vs-liquid-m3-mirage
20:25:12 [INFO] Setting parse rate to 128
20:25:12 [INFO] Go version>=1.14.0
20:25:12 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/Liquid/Mirage/liquid-vs-mad-lions-m2-mirage.dem
20:25:12 [INFO] Setting demo id to liquid-vs-mad-lions-m2-mirage
20:25:12 [INFO] Setting parse rate to 128
20:25:13 [INFO] Go version>=1.14.0
20:25:13 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/Liquid/Mirage/liquid-vs-o

# Demo Parsers: 7


20:25:13 [INFO] Go version>=1.14.0
20:25:13 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/Furia/Mirage/forze-vs-furia-m3-mirage.dem
20:25:13 [INFO] Setting demo id to forze-vs-furia-m3-mirage
20:25:13 [INFO] Setting parse rate to 128
20:25:13 [INFO] Go version>=1.14.0
20:25:13 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/Furia/Mirage/furia-vs-complexity-m1-mirage.dem
20:25:13 [INFO] Setting demo id to furia-vs-complexity-m1-mirage
20:25:13 [INFO] Setting parse rate to 128
20:25:13 [INFO] Go version>=1.14.0
20:25:13 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/Furia/Mirage/furia-vs-complexity-m1-mirage_1.dem
20:25:13 [INFO] Setting demo id to furia-vs-complexity-m1-mirage
20:25:13 [INFO] Setting parse rate to 128
20:25:13 [INFO] Go version>=1.14.0
20:25:13 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/Furia/Mirage/furia

# Demo Parsers: 21


20:25:15 [INFO] Go version>=1.14.0
20:25:15 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/Complexity/Mirage/complexity-vs-fnatic-m2-mirage.dem
20:25:15 [INFO] Setting demo id to complexity-vs-fnatic-m2-mirage
20:25:15 [INFO] Setting parse rate to 128
20:25:15 [INFO] Go version>=1.14.0
20:25:15 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/Complexity/Mirage/complexity-vs-g2-m1-mirage.dem
20:25:15 [INFO] Setting demo id to complexity-vs-g2-m1-mirage
20:25:15 [INFO] Setting parse rate to 128
20:25:15 [INFO] Go version>=1.14.0
20:25:15 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/Complexity/Mirage/complexity-vs-movistar-riders-m2-mirage.dem
20:25:15 [INFO] Setting demo id to complexity-vs-movistar-riders-m2-mirage
20:25:15 [INFO] Setting parse rate to 128
20:25:15 [INFO] Go version>=1.14.0
20:25:15 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_dem

# Demo Parsers: 23


20:25:17 [INFO] Go version>=1.14.0
20:25:17 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/G2/Mirage/faze-vs-g2-m2-mirage_1.dem
20:25:17 [INFO] Setting demo id to faze-vs-g2-m2-mirage
20:25:17 [INFO] Setting parse rate to 128
20:25:17 [INFO] Go version>=1.14.0
20:25:17 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/G2/Mirage/funplus-phoenix-vs-g2-m2-mirage.dem
20:25:17 [INFO] Setting demo id to funplus-phoenix-vs-g2-m2-mirage
20:25:17 [INFO] Setting parse rate to 128
20:25:17 [INFO] Go version>=1.14.0
20:25:17 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/G2/Mirage/furia-vs-g2-m2-mirage.dem
20:25:17 [INFO] Setting demo id to furia-vs-g2-m2-mirage
20:25:17 [INFO] Setting parse rate to 128
20:25:18 [INFO] Go version>=1.14.0
20:25:18 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/mirage/G2/Mirage/furia-vs-g2-m2-mirage_1.dem
20:25:18 

# Demo Parsers: 22


In [31]:
bigKills, bigRounds, bigDamages = generateData(bigParsers)
vpKills, vpRounds, vpDamages = generateData(vpParsers)
natusKills, natusRounds, natusDamages = generateData(natusParsers)
heroicKills, heroicRounds, heroicDamages = generateData(heroicParsers)

21:14:03 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
21:14:03 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/mirage/Big/Mirage/big-vs-complexity-m1-mirage.dem
21:14:25 [INFO] Wrote demo parse output to big-vs-complexity-m1-mirage.json
21:14:25 [INFO] Reading in JSON from big-vs-complexity-m1-mirage.json
21:14:25 [INFO] JSON data loaded, available in the `json` attribute to parser
21:14:25 [INFO] Successfully parsed JSON output
21:14:25 [INFO] Successfully returned JSON output
21:14:25 [INFO] Parsed kills to Pandas DataFrame
21:14:26 [INFO] Parsed rounds to Pandas DataFrame
21:14:26 [INFO] Parsed damages to Pandas DataFrame
21:14:26 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
21:14:26 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/mirage/Big/Mirage/big-vs-envy-m2-mirage.dem
21:14:46 [INFO] Wrote demo parse output to big-vs-envy-m2-mirage.json
21:14:46 [INFO] Reading in JSON from big-vs-envy-m2-mirage.j

21:18:37 [INFO] Reading in JSON from nip-vs-big-m2-mirage.json
21:18:38 [INFO] JSON data loaded, available in the `json` attribute to parser
21:18:38 [INFO] Successfully parsed JSON output
21:18:38 [INFO] Successfully returned JSON output
21:18:38 [INFO] Parsed kills to Pandas DataFrame
21:18:38 [INFO] Parsed rounds to Pandas DataFrame
21:18:38 [INFO] Parsed damages to Pandas DataFrame
21:18:38 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
21:18:38 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/mirage/Big/Mirage/nip-vs-big-m3-mirage.dem
21:18:56 [INFO] Wrote demo parse output to nip-vs-big-m3-mirage.json
21:18:56 [INFO] Reading in JSON from nip-vs-big-m3-mirage.json
21:18:57 [INFO] JSON data loaded, available in the `json` attribute to parser
21:18:57 [INFO] Successfully parsed JSON output
21:18:57 [INFO] Successfully returned JSON output
21:18:57 [INFO] Parsed kills to Pandas DataFrame
21:18:57 [INFO] Parsed rounds to Pandas DataFrame
21:18:57 

21:23:46 [INFO] JSON data loaded, available in the `json` attribute to parser
21:23:46 [INFO] Successfully parsed JSON output
21:23:46 [INFO] Successfully returned JSON output
21:23:46 [INFO] Parsed kills to Pandas DataFrame
21:23:46 [INFO] Parsed rounds to Pandas DataFrame
21:23:46 [INFO] Parsed damages to Pandas DataFrame
21:23:46 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
21:23:46 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/mirage/Natus/Mirage/gambit-vs-natus-vincere-m3-mirage.dem
21:24:10 [INFO] Wrote demo parse output to gambit-vs-natus-vincere-m3-mirage.json
21:24:10 [INFO] Reading in JSON from gambit-vs-natus-vincere-m3-mirage.json
21:24:11 [INFO] JSON data loaded, available in the `json` attribute to parser
21:24:11 [INFO] Successfully parsed JSON output
21:24:11 [INFO] Successfully returned JSON output
21:24:11 [INFO] Parsed kills to Pandas DataFrame
21:24:11 [INFO] Parsed rounds to Pandas DataFrame
21:24:11 [INFO] Parsed damages 

21:28:36 [INFO] Successfully parsed JSON output
21:28:36 [INFO] Successfully returned JSON output
21:28:36 [INFO] Parsed kills to Pandas DataFrame
21:28:36 [INFO] Parsed rounds to Pandas DataFrame
21:28:36 [INFO] Parsed damages to Pandas DataFrame
21:28:36 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
21:28:36 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/mirage/Heroic/Mirage/heroic-vs-c0ntact-m1-mirage_61471.dem
21:29:01 [INFO] Wrote demo parse output to heroic-vs-c0ntact-m1-mirage.json
21:29:01 [INFO] Reading in JSON from heroic-vs-c0ntact-m1-mirage.json
21:29:02 [INFO] JSON data loaded, available in the `json` attribute to parser
21:29:02 [INFO] Successfully parsed JSON output
21:29:02 [INFO] Successfully returned JSON output
21:29:02 [INFO] Parsed kills to Pandas DataFrame
21:29:02 [INFO] Parsed rounds to Pandas DataFrame
21:29:02 [INFO] Parsed damages to Pandas DataFrame
21:29:02 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo

In [32]:
liquidKills, liquidRounds, liquidDamages = generateData(liquidParsers)
furiaKills, furiaRounds, furiaDamages = generateData(furiaParsers)

21:31:30 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
21:31:30 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/mirage/Liquid/Mirage/faze-vs-liquid-m1-mirage.dem
21:31:57 [INFO] Wrote demo parse output to faze-vs-liquid-m1-mirage.json
21:31:57 [INFO] Reading in JSON from faze-vs-liquid-m1-mirage.json
21:31:58 [INFO] JSON data loaded, available in the `json` attribute to parser
21:31:58 [INFO] Successfully parsed JSON output
21:31:58 [INFO] Successfully returned JSON output
21:31:58 [INFO] Parsed kills to Pandas DataFrame
21:31:58 [INFO] Parsed rounds to Pandas DataFrame
21:31:58 [INFO] Parsed damages to Pandas DataFrame
21:31:58 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
21:31:58 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/mirage/Liquid/Mirage/furia-vs-liquid-m3-mirage.dem
21:32:22 [INFO] Wrote demo parse output to furia-vs-liquid-m3-mirage.json
21:32:22 [INFO] Reading in JSON from furia-vs-liquid-m3

21:37:10 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/mirage/Furia/Mirage/furia-vs-g2-m2-mirage.dem
21:37:34 [INFO] Wrote demo parse output to furia-vs-g2-m2-mirage.json
21:37:34 [INFO] Reading in JSON from furia-vs-g2-m2-mirage.json
21:37:34 [INFO] JSON data loaded, available in the `json` attribute to parser
21:37:34 [INFO] Successfully parsed JSON output
21:37:34 [INFO] Successfully returned JSON output
21:37:34 [INFO] Parsed kills to Pandas DataFrame
21:37:34 [INFO] Parsed rounds to Pandas DataFrame
21:37:34 [INFO] Parsed damages to Pandas DataFrame
21:37:34 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
21:37:34 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/mirage/Furia/Mirage/furia-vs-g2-m2-mirage_1.dem
21:37:57 [INFO] Wrote demo parse output to furia-vs-g2-m2-mirage.json
21:37:57 [INFO] Reading in JSON from furia-vs-g2-m2-mirage.json
21:37:58 [INFO] JSON data loaded, available in the `json` attribute to parser
21:37:58 [INF

21:42:51 [INFO] JSON data loaded, available in the `json` attribute to parser
21:42:51 [INFO] Successfully parsed JSON output
21:42:51 [INFO] Successfully returned JSON output
21:42:51 [INFO] Parsed kills to Pandas DataFrame
21:42:51 [INFO] Parsed rounds to Pandas DataFrame
21:42:51 [INFO] Parsed damages to Pandas DataFrame
21:42:51 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
21:42:51 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/mirage/Furia/Mirage/natus-vincere-vs-furia-m1-mirage.dem
21:43:14 [INFO] Wrote demo parse output to natus-vincere-vs-furia-m1-mirage.json
21:43:14 [INFO] Reading in JSON from natus-vincere-vs-furia-m1-mirage.json
21:43:15 [INFO] JSON data loaded, available in the `json` attribute to parser
21:43:15 [INFO] Successfully parsed JSON output
21:43:15 [INFO] Successfully returned JSON output
21:43:15 [INFO] Parsed kills to Pandas DataFrame
21:43:15 [INFO] Parsed rounds to Pandas DataFrame
21:43:15 [INFO] Parsed damages to 

In [33]:
complexityKills, complexityRounds, complexityDamages = generateData(complexityParsers)
g2Kills, g2Rounds, g2Damages = generateData(g2Parsers)

21:44:07 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
21:44:07 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/mirage/Complexity/Mirage/big-vs-complexity-m1-mirage.dem
21:44:30 [INFO] Wrote demo parse output to big-vs-complexity-m1-mirage.json
21:44:30 [INFO] Reading in JSON from big-vs-complexity-m1-mirage.json
21:44:31 [INFO] JSON data loaded, available in the `json` attribute to parser
21:44:31 [INFO] Successfully parsed JSON output
21:44:31 [INFO] Successfully returned JSON output
21:44:31 [INFO] Parsed kills to Pandas DataFrame
21:44:31 [INFO] Parsed rounds to Pandas DataFrame
21:44:31 [INFO] Parsed damages to Pandas DataFrame
21:44:31 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
21:44:31 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/mirage/Complexity/Mirage/complexity-vs-ence-m3-mirage.dem
21:44:56 [INFO] Wrote demo parse output to complexity-vs-ence-m3-mirage.json
21:44:56 [INFO] Reading in JSON 

21:49:07 [INFO] Parsed rounds to Pandas DataFrame
21:49:07 [INFO] Parsed damages to Pandas DataFrame
21:49:07 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
21:49:07 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/mirage/Complexity/Mirage/fnatic-vs-complexity-m2-mirage.dem
21:49:31 [INFO] Wrote demo parse output to fnatic-vs-complexity-m2-mirage.json
21:49:31 [INFO] Reading in JSON from fnatic-vs-complexity-m2-mirage.json
21:49:32 [INFO] JSON data loaded, available in the `json` attribute to parser
21:49:32 [INFO] Successfully parsed JSON output
21:49:32 [INFO] Successfully returned JSON output
21:49:32 [INFO] Parsed kills to Pandas DataFrame
21:49:32 [INFO] Parsed rounds to Pandas DataFrame
21:49:32 [INFO] Parsed damages to Pandas DataFrame
21:49:32 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
21:49:32 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/mirage/Complexity/Mirage/furia-vs-complexity-m1-mirage.de

21:53:42 [INFO] Successfully returned JSON output
21:53:42 [INFO] Parsed kills to Pandas DataFrame
21:53:42 [INFO] Parsed rounds to Pandas DataFrame
21:53:42 [INFO] Parsed damages to Pandas DataFrame
21:53:42 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
21:53:42 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/mirage/G2/Mirage/ence-vs-g2-m2-mirage.dem
21:54:04 [INFO] Wrote demo parse output to ence-vs-g2-m2-mirage.json
21:54:04 [INFO] Reading in JSON from ence-vs-g2-m2-mirage.json
21:54:05 [INFO] JSON data loaded, available in the `json` attribute to parser
21:54:05 [INFO] Successfully parsed JSON output
21:54:05 [INFO] Successfully returned JSON output
21:54:05 [INFO] Parsed kills to Pandas DataFrame
21:54:05 [INFO] Parsed rounds to Pandas DataFrame
21:54:05 [INFO] Parsed damages to Pandas DataFrame
21:54:05 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
21:54:05 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/de

21:59:44 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/mirage/G2/Mirage/g2-vs-mousesports-m3-mirage.dem
22:00:18 [INFO] Wrote demo parse output to g2-vs-mousesports-m3-mirage.json
22:00:18 [INFO] Reading in JSON from g2-vs-mousesports-m3-mirage.json
22:00:18 [INFO] JSON data loaded, available in the `json` attribute to parser
22:00:18 [INFO] Successfully parsed JSON output
22:00:18 [INFO] Successfully returned JSON output
22:00:18 [INFO] Parsed kills to Pandas DataFrame
22:00:18 [INFO] Parsed rounds to Pandas DataFrame
22:00:18 [INFO] Parsed damages to Pandas DataFrame
22:00:18 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
22:00:18 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/mirage/G2/Mirage/g2-vs-nip-m2-mirage.dem
22:01:03 [INFO] Wrote demo parse output to g2-vs-nip-m2-mirage.json
22:01:03 [INFO] Reading in JSON from g2-vs-nip-m2-mirage.json
22:01:04 [INFO] JSON data loaded, available in the `json` attribute to parser
22:01:04 

In [34]:
for x in range(len(heroicKills)):
    if(heroicKills[x].shape[0] < 100):
        print("failed demo " + str(x))

In [35]:
for x in range(len(vpKills)):
    if(vpKills[x].shape[0] < 100):
        print("failed demo " + str(x))

In [36]:
for x in range(len(natusKills)):
    if(natusKills[x].shape[0] < 100):
        print("failed demo " + str(x))

In [37]:
for x in range(len(bigKills)):
    if(bigKills[x].shape[0] < 100):
        print("failed demo " + str(x))

In [38]:
for x in range(len(liquidKills)):
    if(liquidKills[x].shape[0] < 100):
        print("failed demo " + str(x))

In [39]:
for x in range(len(furiaKills)):
    if(furiaKills[x].shape[0] < 100):
        print("failed demo " + str(x))

In [40]:
for x in range(len(complexityKills)):
    if(complexityKills[x].shape[0] < 100):
        print("failed demo " + str(x))

failed demo 4


In [41]:
complexityKills.pop(4)

,Tick,Second,AttackerSteamId,AttackerName,AttackerTeam,AttackerSide,AttackerX,AttackerY,AttackerZ,AttackerAreaId,...,IsHeadshot,IsTrade,PlayerTradedName,PlayerTradedTeam,PlayerTradedSteamId,IsFirstKill,Weapon,RoundNum,MatchId,MapName
0,34831,22.393701,76561198041683378,NiKo,G2 Esports,CT,-1380.287720,149.593201,-164.455460,26,...,True,False,None,None,NaN,True,USP-S,1,complexity-vs-g2-m1-mirage,de_mirage
1,34852,22.559055,76561198024905796,kennyS,G2 Esports,CT,-1252.430786,313.132812,-165.086090,417,...,True,False,poizon,compLexity,7.656120e+16,False,USP-S,1,complexity-vs-g2-m1-mirage,de_mirage
2,35108,24.574803,76561198024905796,kennyS,G2 Esports,CT,-1248.174072,442.842621,-166.627045,25,...,False,False,k0nfig,compLexity,7.656120e+16,False,USP-S,1,complexity-vs-g2-m1-mirage,de_mirage
3,35485,27.543307,76561198024905796,kennyS,G2 Esports,CT,-1294.046875,392.380951,-167.968750,25,...,True,False,jks,compLexity,7.656120e+16,False,USP-S,1,complexity-vs-g2-m1-mirage,de_mirage
4,36492,35.472441,76561198068944951,AmaNEk,G2 Esports,CT,-2263.518555,655.981262,-39.968750,902,...,False,False,blameF,compLexity,7.656120e+16,False,USP-S,1,complexity-vs-g2-m1-mirage,de_mirage


In [42]:
for x in range(len(g2Kills)):
    if(g2Kills[x].shape[0] < 100):
        print("failed demo " + str(x))

failed demo 0


In [43]:
g2Kills.pop(0)

,Tick,Second,AttackerSteamId,AttackerName,AttackerTeam,AttackerSide,AttackerX,AttackerY,AttackerZ,AttackerAreaId,...,IsHeadshot,IsTrade,PlayerTradedName,PlayerTradedTeam,PlayerTradedSteamId,IsFirstKill,Weapon,RoundNum,MatchId,MapName
0,34831,22.393701,76561198041683378,NiKo,G2 Esports,CT,-1380.287720,149.593201,-164.455460,26,...,True,False,None,None,NaN,True,USP-S,1,complexity-vs-g2-m1-mirage,de_mirage
1,34852,22.559055,76561198024905796,kennyS,G2 Esports,CT,-1252.430786,313.132812,-165.086090,417,...,True,False,poizon,compLexity,7.656120e+16,False,USP-S,1,complexity-vs-g2-m1-mirage,de_mirage
2,35108,24.574803,76561198024905796,kennyS,G2 Esports,CT,-1248.174072,442.842621,-166.627045,25,...,False,False,k0nfig,compLexity,7.656120e+16,False,USP-S,1,complexity-vs-g2-m1-mirage,de_mirage
3,35485,27.543307,76561198024905796,kennyS,G2 Esports,CT,-1294.046875,392.380951,-167.968750,25,...,True,False,jks,compLexity,7.656120e+16,False,USP-S,1,complexity-vs-g2-m1-mirage,de_mirage
4,36492,35.472441,76561198068944951,AmaNEk,G2 Esports,CT,-2263.518555,655.981262,-39.968750,902,...,False,False,blameF,compLexity,7.656120e+16,False,USP-S,1,complexity-vs-g2-m1-mirage,de_mirage


In [44]:
liquidKill = concat_data(liquidKills)
liquidRound = concat_data(liquidRounds)
liquidDamage = concat_data(liquidDamages)

In [45]:
for x in range(len(liquidKill)):
    if  liquidKill.iloc[x]['AttackerAreaName'] != None:
        liquidKill.at[x, 'VictimAreaName'] = mappedPlaceNames[liquidKill.iloc[x]['VictimAreaName']]
        liquidKill.at[x, 'AttackerAreaName'] = mappedPlaceNames[liquidKill.iloc[x]['AttackerAreaName']]

for x in range(len(liquidDamage)):
    if  liquidDamage.iloc[x]['AttackerAreaName'] != None:
        liquidDamage.at[x, 'VictimAreaName'] = mappedPlaceNames[liquidDamage.iloc[x]['VictimAreaName']]
        liquidDamage.at[x, 'AttackerAreaName'] = mappedPlaceNames[liquidDamage.iloc[x]['AttackerAreaName']]

In [46]:
liquidKill.AttackerName.value_counts().index

Index(['EliGE', 'Stewie2K', 'NAF', 'Grim', 'FalleN', 'Twistzz', 's1mple',
       'Jame', 'YEKINDAR', 'acoR', 'Perfecto', 'electronic', 'coldzera',
       'misutaaa', 'flamie', 'sjuush', 'VINI', 'roeJ', 'yuurih', 'valde',
       'HEN1', 'arT', 'broky', 'HooXi', 'Boombl4', 'buster', 'RpK', 'refrezh',
       'SANJI', 'mantuu', 'KSCERATO', 'apEX', 'ZywOo', 'shox', 'ISSAA', 'rain',
       'qikert', 'Aleksib', 'karrigan', 'NBK-'],
      dtype='object')

In [47]:
liquid_names = {'Elige' : ['EliGE'], 'Stewie2k' : ['Stewie2K'], 'Naf-fly' : ['NAF'], 'Grim' : ['Grim'],
            'Fallen' : ['FalleN']}

In [48]:
furiaKill = concat_data(furiaKills)
furiaRound = concat_data(furiaRounds)
furiaDamage = concat_data(furiaDamages)

In [49]:
for x in range(len(furiaKill)):
    if  furiaKill.iloc[x]['AttackerAreaName'] != None:
        furiaKill.at[x, 'VictimAreaName'] = mappedPlaceNames[furiaKill.iloc[x]['VictimAreaName']]
        furiaKill.at[x, 'AttackerAreaName'] = mappedPlaceNames[furiaKill.iloc[x]['AttackerAreaName']]

for x in range(len(furiaDamage)):
    if  furiaDamage.iloc[x]['AttackerAreaName'] != None:
        furiaDamage.at[x, 'VictimAreaName'] = mappedPlaceNames[furiaDamage.iloc[x]['VictimAreaName']]
        furiaDamage.at[x, 'AttackerAreaName'] = mappedPlaceNames[furiaDamage.iloc[x]['AttackerAreaName']]

In [50]:
furiaKill.AttackerName.value_counts().index

Index(['KSCERATO', 'yuurih', 'arT', 'VINI', 'junior', 'HEN1', 'huNter-',
       'nexa', 'kennyS', 'NiKo ', 'AmaNEk', 'blameF', 's1mple', 'YEKINDAR',
       'stavn', 'electronic', 'Jame', 'NiKo', 'poizon', 'jks', 'buster',
       'Hobbit', 'Ax1Le', 'SANJI', 'honda', 'k0nfig', 'qikert', 'cadiaN',
       'TeSeS', 'nafany', 'Boombl4', 'chelo', 'sh1ro', 'sh1r0', 'RUSH',
       'niko ', 'Perfecto', 'Brehze', 'Jerry x LUKOIL', 'NBK-', 'refrezh',
       'exit', 'mantuu', 'valde', 'tarik', 'CeRq', 'Aleksib', 'Twistzz',
       'interz', 'yel', 'biguzera', 'zorte x LUKOIL', 'FL1T x LUKOIL',
       'stanislaw', 'Grim', 'KENSi x LUKOIL', 'saffee', 'almazer x LUKOIL',
       'Stewie2K', 'Ethan', 'boltz', 'shz', 'hardzao', 'rain', 'b0RUP',
       'JACKZ', 'flamie', 'ISSAA', 'PKL', 'sjuush', 'broky', 'coldzera',
       'NEKiZ', 'EliGE', 'b1t', 'NAF', 'karrigan'],
      dtype='object')

In [51]:
furia_names = {'KSCERATO' : ['KSCERATO'], 'Yuurih' : ['yuurih'], 'Art' : ['arT'], 'Vini' : ['VINI'],
            'Junior' : ['junior']}

In [52]:
complexityKill = concat_data(complexityKills)
complexityRound = concat_data(complexityRounds)
complexityDamage = concat_data(complexityDamages)

In [53]:
for x in range(len(complexityKill)):
    if  complexityKill.iloc[x]['AttackerAreaName'] != None:
        complexityKill.at[x, 'VictimAreaName'] = mappedPlaceNames[complexityKill.iloc[x]['VictimAreaName']]
        complexityKill.at[x, 'AttackerAreaName'] = mappedPlaceNames[complexityKill.iloc[x]['AttackerAreaName']]

for x in range(len(complexityDamage)):
    if  complexityDamage.iloc[x]['AttackerAreaName'] != None:
        complexityDamage.at[x, 'VictimAreaName'] = mappedPlaceNames[complexityDamage.iloc[x]['VictimAreaName']]
        complexityDamage.at[x, 'AttackerAreaName'] = mappedPlaceNames[complexityDamage.iloc[x]['AttackerAreaName']]

In [54]:
complexityKill.AttackerName.value_counts().index

Index(['k0nfig', 'blameF', 'poizon', 'jks', 'RUSH', 'ZywOo', 'KRIMZ', 's1mple',
       'ropz', 'Bymas', 'k1to', 'RpK', 'Brollan', 'electronic', 'Boombl4',
       'mantuu', 'acoR', 'apEX', 'cadiaN', 'Perfecto', 'valde', 'hampus',
       'misutaaa', 'shokz', 'Plopski', 'Golden', 'junior', 'VINI', 'KSCERATO',
       'ISSAA', 'nawwk', 'JW', 'interz', 'mopoz', 'smooya', 'FROZENN',
       'frozen', 'refrezh', 'REZ', 'ztr', 'Aleksib', 'yuurih', 'niko ',
       'Jamppi', 'alex', 'buster', 'tiziaN', 'INS', 'qikert', 'XANTARES',
       'shox ', 'stavn', 'malta', 'doto', 'aliStair', 'broky', 'b1t', 'slaxz-',
       'syrsoN', 'arT', 'HObbit', 'allu', 'SADDYX', 'faveN', 'Kjaerbye',
       'JUGi', 'YEKINDAR', 'nafany', 'tabseN', 'sjuush', 'TeSeS', 'flamie',
       'sh1ro', 'flusha', 'Ax1Le', 'SANJI', 'steel', 'Sico', 'stanislaw',
       'Jame', 'coldzera', 'shox', 'ddexterrr', 'Ethan', 'Jackinho', 'suNny',
       'Spiidi', 'olofmeister', 'Brehze', 'kressy', 'denis', 'rain', 'dexter',
       'tarik',

In [55]:
complexity_names = {'Konfig' : ['k0nfig'], 'BlameF' : ['blameF'], 'Poizon' : ['poizon'], 'Jks' : ['jks'],
            'Rush' : ['RUSH']}

In [56]:
g2Kill = concat_data(g2Kills)
g2Round = concat_data(g2Rounds)
g2Damage = concat_data(g2Damages)

In [57]:
for x in range(len(g2Kill)):
    if  g2Kill.iloc[x]['AttackerAreaName'] != None:
        g2Kill.at[x, 'VictimAreaName'] = mappedPlaceNames[g2Kill.iloc[x]['VictimAreaName']]
        g2Kill.at[x, 'AttackerAreaName'] = mappedPlaceNames[g2Kill.iloc[x]['AttackerAreaName']]

for x in range(len(g2Damages)):
    if  g2Damage.iloc[x]['AttackerAreaName'] != None:
        g2Damage.at[x, 'VictimAreaName'] = mappedPlaceNames[g2Damage.iloc[x]['VictimAreaName']]
        g2Damage.at[x, 'AttackerAreaName'] = mappedPlaceNames[g2Damage.iloc[x]['AttackerAreaName']]

In [58]:
g2Kill.AttackerName.value_counts().index

Index(['huNter-', 'nexa', 'NiKo', 'AmaNEk', 'JACKZ', 'kennyS', 'NiKo ',
       'yuurih', 'arT', 'KSCERATO', 'degster', 'VINI', 'frozen', 'acoR', 'sdy',
       'mir', 'ropz', 'REZ', 'nawwk', 'tabseN', 'chopper', 'Bymas', 'hampus',
       'HEN1', 'tiziaN', 'dexter', 'syrsoN', 'XANTARES', 'coldzera', 'magixx',
       'Plopski', 'broky', 'ZywOo', 'Ax1Le', 'k1to', 'rain', 'Twistzz',
       's1mple', 'misutaaa', 'ztr', 'Perfecto', 'SH1R0', 'flamie', 'apEX',
       'honda', 'electronic', 'Kjaerbye', 'Boombl4', 'interz', 'Snappi',
       'Hobbit', 'doto', 'zehN', 'shox', 'twist', 'nafany', 'RpK', 'maden',
       'junior', 'Spinx', 'dycha', 'olofmeister', 'karrigan', 'farlig',
       'STYKO', 'allu', 'emi'],
      dtype='object')

In [59]:
g2_names = {'Hunter' : ['huNter-'], 'Nexa' : ['nexa'], 'Niko' : ['NiKo ', 'NiKo'], 'Amanek' : ['AmaNEk'],
            'Jackz' : ['JACKZ']}

In [60]:
bigKill = concat_data(bigKills)
bigRound = concat_data(bigRounds)
bigDamage = concat_data(bigDamages)

In [61]:
for x in range(len(bigKill)):
    if  bigKill.iloc[x]['AttackerAreaName'] != None:
        bigKill.at[x, 'VictimAreaName'] = mappedPlaceNames[bigKill.iloc[x]['VictimAreaName']]
        bigKill.at[x, 'AttackerAreaName'] = mappedPlaceNames[bigKill.iloc[x]['AttackerAreaName']]

for x in range(len(bigDamage)):
    if  bigDamage.iloc[x]['AttackerAreaName'] != None:
        bigDamage.at[x, 'VictimAreaName'] = mappedPlaceNames[bigDamage.iloc[x]['VictimAreaName']]
        bigDamage.at[x, 'AttackerAreaName'] = mappedPlaceNames[bigDamage.iloc[x]['AttackerAreaName']]

In [62]:
vpKill = concat_data(vpKills)
vpRound = concat_data(vpRounds)
vpDamage = concat_data(vpDamages)

In [63]:
for x in range(len(vpKill)):
    if  vpKill.iloc[x]['AttackerAreaName'] != None:
        vpKill.at[x, 'VictimAreaName'] = mappedPlaceNames[vpKill.iloc[x]['VictimAreaName']]
        vpKill.at[x, 'AttackerAreaName'] = mappedPlaceNames[vpKill.iloc[x]['AttackerAreaName']]

for x in range(len(vpDamage)):
    if  vpDamage.iloc[x]['AttackerAreaName'] != None:
        vpDamage.at[x, 'VictimAreaName'] = mappedPlaceNames[vpDamage.iloc[x]['VictimAreaName']]
        vpDamage.at[x, 'AttackerAreaName'] = mappedPlaceNames[vpDamage.iloc[x]['AttackerAreaName']]

In [64]:
natusKill = concat_data(natusKills)
natusRound = concat_data(natusRounds)
natusDamage = concat_data(natusDamages)

In [65]:
for x in range(len(natusKill)):
    if  natusKill.iloc[x]['AttackerAreaName'] != None:
        natusKill.at[x, 'VictimAreaName'] = mappedPlaceNames[natusKill.iloc[x]['VictimAreaName']]
        natusKill.at[x, 'AttackerAreaName'] = mappedPlaceNames[natusKill.iloc[x]['AttackerAreaName']]

for x in range(len(natusDamage)):
    if  natusDamage.iloc[x]['AttackerAreaName'] != None:
        natusDamage.at[x, 'VictimAreaName'] = mappedPlaceNames[natusDamage.iloc[x]['VictimAreaName']]
        natusDamage.at[x, 'AttackerAreaName'] = mappedPlaceNames[natusDamage.iloc[x]['AttackerAreaName']]

In [66]:
heroicKill = concat_data(heroicKills)
heroicRound = concat_data(heroicRounds)
heroicDamage = concat_data(heroicDamages)

In [67]:
for x in range(len(heroicKill)):
    if heroicKill.iloc[x]['AttackerAreaName'] != None and heroicKill.iloc[x]['AttackerAreaName'] != 'Tunnel1':
        heroicKill.at[x, 'VictimAreaName'] = mappedPlaceNames[heroicKill.iloc[x]['VictimAreaName']]
        heroicKill.at[x, 'AttackerAreaName'] = mappedPlaceNames[heroicKill.iloc[x]['AttackerAreaName']]

for x in range(len(heroicDamage)):
    if  heroicDamage.iloc[x]['AttackerAreaName'] != None:
        heroicDamage.at[x, 'VictimAreaName'] = mappedPlaceNames[heroicDamage.iloc[x]['VictimAreaName']]
        heroicDamage.at[x, 'AttackerAreaName'] = mappedPlaceNames[heroicDamage.iloc[x]['AttackerAreaName']]

In [68]:
big_names = {'Syrson' : ['syrsoN', 'BIG syrsoN'], 'K1to' : ['k1to', 'BIG k1to'], 'Tabsen' : ['tabseN', 'BIG tabseN'], 'Tizian' : ['tiziaN', 'BIG tiziaN'],
            'Xantares' : ['XANTARES', 'BIG XANTARES']}

In [69]:
vp_names = {'Jame' : ['Jame', 'Jame . Parimatch'], 'Buster' : ['buster', 'buster . Parimatch'], 'Yekindar' : ['YEKINDAR', 'YEKINDAR . Parimatch'],
            'Sanji' : ['SANJI', 'SANJI . Parimatch'],  'Qikert' : ['qikert', 'Qikert', 'qikert . Parimatch']}

In [70]:
natus_names = {'Simple' : ['s1mple'], 'Electronic' : ['electronic '], 'Boombl4' : ['Boombl4'],
            'Perfecto' : ['Perfecto'],  'B1t' : ['b1t'], 'Flamie' : ['flamie']}

In [71]:
heroic_names = {'Stavn' : ['stavn'], 'Teses' : ['TeSeS'], 'Cadian' : ['cadiaN'],
            'Refresh' : ['refrezh'],  'Sjuush' : ['sjuush'], 'B0rup' : ['b0RUP']}

In [76]:
bigDifferentials = createSidedCSVDF(big_names, bigDamage, bigKill, bigRound)

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Boolean Serie

In [77]:
bigDifferentials.to_csv(big_output_path)

In [78]:
liquidDifferentials = createSidedCSVDF(liquid_names, liquidDamage, liquidKill, liquidRound)

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Boolean Serie

In [79]:
liquidDifferentials.to_csv(liquid_output_path)

In [80]:
furiaDifferentials = createSidedCSVDF(furia_names, furiaDamage, furiaKill, furiaRound)

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Boolean Serie

In [81]:
furiaDifferentials.to_csv(furia_output_path)

In [82]:
g2Differentials = createSidedCSVDF(g2_names, g2Damage, g2Kill, g2Round)

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Boolean Serie

In [83]:
g2Differentials.to_csv(g2_output_path)

In [84]:
complexityDifferentials = createSidedCSVDF(complexity_names, complexityDamage, complexityKill, complexityRound)

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Boolean Serie

In [85]:
complexityDifferentials.to_csv(complexity_output_path)

In [86]:
vpDifferentials = createSidedCSVDF(vp_names, vpDamage, vpKill, vpRound)

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Boolean Serie

In [87]:
vpDifferentials.to_csv(vp_output_path)

In [88]:
natusDifferentials = createSidedCSVDF(natus_names, natusDamage, natusKill, natusRound)

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Boolean Serie

In [89]:
natusDifferentials.to_csv(natus_output_path)

In [90]:
heroicDifferentials = createSidedCSVDF(heroic_names, heroicDamage, heroicKill, heroicRound)

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Boolean Serie

In [91]:
heroicDifferentials

,Location,Player,Kills,Deaths,K/D,Damage Dealt,Damage Taken,Damage Differential,Side,Total Rounds
0,CT Side Apps,Stavn,4,7,-3,476,669,-193,CT,358
1,CT Side Apps,Stavn,9,4,5,965,721,244,T,358
2,Top Balcony,Stavn,1,4,-3,106,327,-221,CT,358
3,Top Balcony,Stavn,2,6,-4,211,624,-413,T,358
4,Ticketbooth,Stavn,16,5,11,1659,628,1031,CT,358
...,...,...,...,...,...,...,...,...,...,...
913,T Deep Apps Underpass/Apps Peek,B0rup,2,1,1,298,150,148,T,186
914,Left Side corner deep jungle,B0rup,1,0,1,90,78,12,CT,186
915,Left Side corner deep jungle,B0rup,0,1,-1,24,78,-54,T,186
916,Balcony Boaxes,B0rup,0,0,0,55,0,55,CT,186


In [92]:
heroicDifferentials.to_csv(heroic_output_path)